In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
## from keras import applications `

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head() 

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop non-beneficial ID columns
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])

In [3]:
# APPLICATION_TYPE value counts for binning
application_values = application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_application = application_values[application_values < 1000].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app, 'Other')

In [4]:
# CLASSIFICATION value counts for binning
classification_values = application_df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_class = classification_values[classification_values < 1000].index.tolist()

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls, 'Other') 

In [5]:
# ASK_AMT bins
ask_values_0 = application_df['ASK_AMT'][application_df['ASK_AMT'] == 5000]
ask_values_1 = application_df['ASK_AMT'][(application_df['ASK_AMT'] > 5000)]

# Replace in dataframe
for amt in ask_values_0:
    application_df.ASK_AMT = application_df.ASK_AMT.replace(amt, '5000')
for amt in ask_values_1:
    application_df.ASK_AMT = application_df.ASK_AMT.replace(amt, '>5000') 

In [6]:
# AFFILIATION value counts for binning
affiliation_values = application_df['AFFILIATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_affiliation = affiliation_values[affiliation_values < 1000].index.tolist()

# Replace in dataframe
for aff in replace_affiliation:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(aff, 'Other')

In [7]:
# ORGANIZATION value counts for binning
organization_values = application_df['ORGANIZATION'].value_counts()

# Determine which values to replace if counts are less than 1000
replace_organization = organization_values[organization_values < 1000].index.tolist()

# Replace in dataframe
for org in replace_organization:
    application_df.ORGANIZATION = application_df.ORGANIZATION.replace(org, 'Other') 

In [8]:
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat) 

In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat, 1) 

In [11]:
# Split the preprocessed data into features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78) 

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test) 

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 18

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hiZdden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               4200      
                                                                 
 dense_1 (Dense)             (None, 50)                6050      
                                                                 
 dense_2 (Dense)             (None, 18)                918       
                                                                 
 dense_3 (Dense)             (None, 1)                 19        
                                                                 
Total params: 11,187
Trainable params: 11,187
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('optimized_checkpoints/', exist_ok=True)
checkpoint_path = 'optimized_checkpoints/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000) 

In [16]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=200, callbacks=[cp_callback])

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.6044 - accuracy: 0.6909
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5759 - accuracy: 0.7192
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5687 - accuracy: 0.7236
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5661 - accuracy: 0.7269
Epoch 5/200
773/804 [===========================>..] - ETA: 0s - loss: 0.5667 - accuracy: 0.7250
Epoch 00005: saving model to optimized_checkpoints\weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5656 - accuracy: 0.7259
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5625 - accuracy: 0.7295
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5618 - accuracy: 0.7296
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5597 - accuracy: 0.7316
Epoch 9/200
804/804 [===================

804/804 [==============================] - 1s 2ms/step - loss: 0.5472 - accuracy: 0.7347
Epoch 64/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7360
Epoch 65/200
523/804 [==================>...........] - ETA: 0s - loss: 0.5461 - accuracy: 0.7384
Epoch 00065: saving model to optimized_checkpoints\weights.65.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7354
Epoch 66/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7350
Epoch 67/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7352
Epoch 68/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5469 - accuracy: 0.7341
Epoch 69/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7354
Epoch 70/200
507/804 [=================>............] - ETA: 0s - loss: 0.5439 - accuracy: 0.7366
Epoch 00070: saving model to optimized_checkpoint

804/804 [==============================] - 1s 2ms/step - loss: 0.5427 - accuracy: 0.7376
Epoch 125/200
288/804 [=========>....................] - ETA: 0s - loss: 0.5480 - accuracy: 0.7288
Epoch 00125: saving model to optimized_checkpoints\weights.125.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5423 - accuracy: 0.7375
Epoch 126/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5425 - accuracy: 0.7378
Epoch 127/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5423 - accuracy: 0.7370
Epoch 128/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5424 - accuracy: 0.7378
Epoch 129/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5420 - accuracy: 0.7379
Epoch 130/200
268/804 [=========>....................] - ETA: 1s - loss: 0.5404 - accuracy: 0.7394
Epoch 00130: saving model to optimized_checkpoints\weights.130.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5419 - accu

 58/804 [=>............................] - ETA: 1s - loss: 0.5398 - accuracy: 0.7381
Epoch 00185: saving model to optimized_checkpoints\weights.185.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5393 - accuracy: 0.7393
Epoch 186/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5390 - accuracy: 0.7398
Epoch 187/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5388 - accuracy: 0.7400
Epoch 188/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5390 - accuracy: 0.7393
Epoch 189/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5391 - accuracy: 0.7400
Epoch 190/200
 25/804 [..............................] - ETA: 1s - loss: 0.5384 - accuracy: 0.7462
Epoch 00190: saving model to optimized_checkpoints\weights.190.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5390 - accuracy: 0.7396
Epoch 191/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5389 - accu

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')